# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    
    gini = 1.0
    for i in range(len(df[label].unique())):
        gini -= (df[label].value_counts()[i]/len(df))**2  ## 1 - sum(p^2)
    
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    unique = df[attribute].unique()
    result = []
    count = 1
    while count<=int(len(unique)/2)+1:  ## 조합으로 뽑을 class 개수, 조합은 순서가 상관없으니 절반만 반복하면 된다.
        tmp = np.array(list(combinations(unique,count))).tolist()  ## count개 뽑는 조합
        for i in range(len(tmp)):
            result.append(tmp[i])
        count += 1
    return result

In [8]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [130]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    splits = get_binary_split(df, attribute)
    size = len(df)
    
    for split in splits:
        if len(split)>1:  ## 분류된 class가 1개보다 많을 경우 key 이름을 '_'로 이어주기 위해 if로 나눠서 진행
            name = ''
            for j in range(len(split)):
                name+=split[j]
                if j+1!=len(split):name+='_'
            ## gini index는 이진분류로 이분할된 dataframe에서 (get_gini)*(해당 class 비율)을 더해준 값
            result[name] = (get_gini(df[df[attribute].isin(split)], label)*sum(df[attribute].value_counts()[split])+
            get_gini(df[~df[attribute].isin(split)], label)*(size-sum(df[attribute].value_counts()[split])))/size
        else:
            name=split[0]
            result[name] = (get_gini(df[df[attribute]==name], label)*int(df[attribute].value_counts()[name])+
            get_gini(df[df[attribute]!=name], label)*(size-int(df[attribute].value_counts()[name])))/size
            
        
        
    return result

In [133]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.39365079365079364,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [134]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [135]:
##문제1 답안
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [160]:
##문제2 답안
gini_array = {}
for i in pd_data.columns:
    if i=='class_buys_computer':break
    gini_array[i] = min(get_attribute_gini_index(pd_data, i, "class_buys_computer").items())
gini_array

{'age': ('middle_aged', 0.35714285714285715),
 'income': ('high', 0.4428571428571429),
 'student': ('no', 0.3673469387755103),
 'credit_rating': ('excellent', 0.42857142857142855)}

age의 gini index가 0.3571로 변수 중에 가장 작다.

In [166]:
##문제3 답안
df1 = pd_data[pd_data.age=='middle_aged']
df1_gini = {}
for i in df1.columns:
    if i=='class_buys_computer':break
    df1_gini[i] = min(get_attribute_gini_index(df1, i, "class_buys_computer").items())
print(df1_gini,'\n')

df2 = pd_data[pd_data.age!='middle_aged']
df2_gini = {}
for i in df2.columns:
    if i=='class_buys_computer':break
    df2_gini[i] = min(get_attribute_gini_index(df2, i, "class_buys_computer").items())
print(df2_gini)

{'age': ('middle_aged', 0.0), 'income': ('high', 0.0), 'student': ('no', 0.0), 'credit_rating': ('excellent', 0.0)} 

{'age': ('senior', 0.48), 'income': ('high', 0.375), 'student': ('no', 0.31999999999999984), 'credit_rating': ('excellent', 0.4166666666666667)}


middle_aged 데이터프레임은 class_buys_computer가 모두 yes라 get_gini(=1-sum(p^2)가 항상 0으로 나와 모든 gini index가 0으로 나온다.

youth와 senior 데이터프레임에서는 student의 gini index가 0.3199로 가장 작다.